In [ ]:
# import the necessary packages
import torch
import torchvision
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
from torch.utils.data import DataLoader, random_split
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from Models.LeNet import LeNet
from torchsummary import summary
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import os
from datetime import datetime

In [ ]:
# Functions
def guardar_figura(figura, nombre_base="confusion_matrix", carpeta="Resultados_Graficos", dpi=500):
    """
    Guarda la figura en la carpeta especificada con nombre basado en timestamp.
    
    Parámetros:
    - figura: objeto matplotlib Figure
    - nombre_base: nombre base del archivo
    - carpeta: carpeta donde guardar
    - dpi: resolución de salida
    """
    os.makedirs(carpeta, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nombre_archivo = f"{nombre_base}_{timestamp}.pdf"
    ruta_completa = os.path.join(carpeta, nombre_archivo)
    
    figura.savefig(ruta_completa, dpi=dpi, bbox_inches="tight")
    plt.close(figura)
    return ruta_completa

In [ ]:
# Configure hyperparameters
BATCH_SIZE = 32
EPOCHS     = 20
LR         = 1e-3

TRAIN_SPLIT = 0.90
VAL_SPLIT   = 0.10

# set the device we will  be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'[INFO] Device used: {device}')

print(f'[INFO] Loading datasets....')
fullTrainData = KMNIST(root = "data", train = True, download = False, transform = ToTensor())
TestData      = KMNIST(root = "data", train = False, download = False, transform = ToTensor())

counts = Counter(fullTrainData.targets.tolist())
print(f'[INFO] Train Data Info:')
for idx, class_name in enumerate(fullTrainData.classes):
    print(f'Class {idx+1 :2d} ({class_name}):\t {counts[idx]} images')

print(f'[INFO] Generating the train/val/test datasets')
totalSamples  = len(fullTrainData)
nTrainSamples = int(TRAIN_SPLIT*totalSamples)
nValSamples   = totalSamples-nTrainSamples

(trainData, valData) = random_split(
    fullTrainData, 
    [nTrainSamples, nValSamples],
    generator = torch.Generator().manual_seed(42)
)

# create the dataloaders
trainDataLoader = DataLoader(trainData, batch_size = BATCH_SIZE, shuffle = True)
valDataLoader   = DataLoader(valData, batch_size = BATCH_SIZE)
testDataLoader  = DataLoader(TestData, batch_size = BATCH_SIZE)

n_classes       = len(trainData.dataset.classes)

print(f'[INFO] Total images train/val: {nTrainSamples, nValSamples}')
print(f'[INFO] Total images test: {len(TestData)}')
print(f'[INFO] Num of classes: {n_classes}')
print(f'[INFO] Size of image: {trainData[0][0].shape}')

In [ ]:
# Visualize the images dataset
fig = plt.figure(figsize = (20, 15))
for i in range(12):
    img, label = valDataLoader.dataset[i]
    ax = fig.add_subplot(3, 4, i+1)
    ax.imshow(img.squeeze(0), cmap='gray')
    ax.set_title(f'Class {label}: {testDataLoader.dataset.classes[label]}')
    ax.set_axis_off()
plt.tight_layout()
plt.show()

In [ ]:
# Initializing the model
print(f'[INFO] Initializing the model (LeNet) ...')
model  = LeNet(nChannels = 1, nclasses = n_classes).to(device)
opt    = Adam(model.parameters(), lr = LR)
lossFn = CrossEntropyLoss()

history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

summary(model, input_size = trainDataLoader.dataset[0][0].shape)

In [ ]:
print(f'[INFO] Training the model...')

for i in range(0, EPOCHS):
    model.train()

    total_train_loss = 0.0
    total_val_loss   = 0.0

    train_correct    = 0.0
    val_correct      = 0.0

    for img, label in trainDataLoader:
        (img, label) = (img.to(device), label.to(device))
        pred         = model(img)
        loss         = lossFn(pred, label)

        opt.zero_grad()
        loss.backward()
        opt.step()

        total_train_loss += loss
        train_correct    += (pred.argmax(1) == label).type(torch.float).sum().item()
    
    with torch.no_grad():
        model.eval()

        for img, label in valDataLoader:
            (img, label) = (img.to(device), label.to(device))
            pred         = model(img)

            total_val_loss += lossFn(pred, label)
            val_correct    += (pred.argmax(1) == label).type(torch.float).sum().item()
    
    mean_train_loss = total_train_loss / (len(trainDataLoader.dataset)//BATCH_SIZE)
    mean_val_loss   = total_val_loss / (len(valDataLoader.dataset)//BATCH_SIZE)

    train_correct   = train_correct / (len(trainDataLoader.dataset))
    val_correct     = val_correct / (len(valDataLoader.dataset))

    history["train_loss"].append(mean_train_loss.cpu().detach().numpy())
    history["train_acc"].append(train_correct)
    history["val_loss"].append(mean_val_loss.cpu().detach().numpy())
    history["val_acc"].append(val_correct)

    print(f'[INFO] Epoch: {i+1}/{EPOCHS}')
    print(f'\t Train loss: {mean_train_loss:.6f}, \t Train acc: {train_correct:.6f}')
    print(f'\t Val loss: {mean_val_loss:.6f}, \t Val acc: {val_correct:.6f}')

In [ ]:
plt.style.use("ggplot")
# ------------------------------------------------------------------
# plot the training and validation accuracy
# ------------------------------------------------------------------
plt.figure()
plt.plot(history["train_acc"], color = 'r', label = "Training accuracy")
plt.plot(history["val_acc"], color = 'b', label = "Validation accuracy")
plt.title("Training & Validation Accuracy")
plt.legend(loc = 0)
plt.grid(True)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()
# ------------------------------------------------------------------
# plot the training and validation loss
# ------------------------------------------------------------------
plt.figure()
plt.plot(history["train_loss"], color = 'r', label = "Training loss")
plt.plot(history["val_loss"], color = 'b', label = "Validation loss")
plt.title("Training & Validation Loss")
plt.legend(loc = 0)
plt.grid(True)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

print(f'[INFO] Evaluating the model...')

with torch.no_grad():
    model.eval()
    probs = []
    true_labels = []

    for x, y in testDataLoader:
        x = x.to(device)
        y = y.to(device)

        outputs = model(x)
        preds   = outputs.argmax(dim=1)
        probs.extend(preds.cpu().numpy())
        true_labels.extend(y.cpu().numpy())

print(
    classification_report(
    true_labels, np.array(probs), target_names=TestData.classes
    )
)

fig, ax= plt.subplots(figsize=(10, 10))
ConfusionMatrixDisplay.from_predictions(
        true_labels, probs, display_labels=TestData.classes, xticks_rotation="vertical",
        normalize="pred",
        ax=ax, colorbar=False, cmap="Blues", values_format='.2f')
plt.tick_params(axis="both", labelsize = 16)
plt.ylabel("Clase Real", fontsize=18)
plt.xlabel("Clase Predicha", fontsize=18)
plt.title("Confusion Matrix normalized (%)", fontsize = 20)
plt.grid(False)
plt.tight_layout()

In [ ]:
guardar_figura(fig)